In [ ]:
# imports
import textract
import string
import re
from pathlib import Path
import requests
import os

# download pdf from url

url = "http://arxiv.org/pdf/1411.6753v1"
url_name = url.split("/")[-1]

pdf = Path(url_name+".pdf")
response = requests.get(url)
pdf.write_bytes(response.content)

try:
    text = textract.process(pdf)
except:
    # this method takes much longer... either keep this here or replace with error message for user
    text = textract.process(pdf, method="tesseract", language="eng")
# decode text string from pdf
text = text.decode('utf-8')
# split at double breaks
text = text.split("\n\n")
# collect abstract text
abstract = []
cnt = 0
while cnt < len(text):
    if "abstract" in text[cnt].lower():
        if "abstract" in text[cnt].lower() and len(text[cnt].split()) < 10:
            cnt += 1
        while True:
            if "*" in text[cnt] and len(text[cnt].split()) < 10:
                cnt += 1
            elif len(text[cnt].split()) > 10:
                if "introduction" in text[cnt][:20].lower() or "i ntroduction" in text[cnt][:20].lower():
                    break
                abstract.append(text[cnt])
                cnt +=1
            elif len(text[cnt].split()) == 1:
                try:
                    int(text[cnt])
                except:
                    break
                break
            else:
                break
        break
    else:
        cnt += 1
if not abstract:
    stop = 0
    while True:
        if "introduction" in text[stop].lower() and len(text[stop].split()) < 5:
            break
        else:
            stop += 1
    cnt = 0
    while cnt < stop:
        if text[cnt] == " " or len(text[cnt]) < 2:
            if len(text[cnt+1].split()) < 10:
                cnt += 1
            else:
                abstract.append(text[cnt+1])
                if text[cnt+1][-1] == "-" or text[cnt+1][:-1] == "-":
                    while True:
                        if text[cnt+2][0].islower():
                            abstract.append(text[cnt+2])
                            break
                        else:
                            cnt += 1
        cnt += 1
# format abstract to present to user
abstract_string = ""
for a in abstract:
    a = re.sub("-\n", "", a)
    a = re.sub("\n", " ", a)
    if a[-1] not in string.punctuation:
        abstract_string += a+" "
    elif a[-1] == "-" or a[-1] == "-":
        abstract_string += a[:-1]
    elif a[-1] in string.punctuation:
        abstract_string += a+" "
# remove word 'abstract' and date at start
if "abstract" in abstract_string[:8].lower():
    abstract_string = abstract_string[8:]
dummy1 = abstract_string.split(". ")
dummy2 = abstract_string.split()
if "abstract" in dummy1[0].lower():
    cnt = 0
    while True:
        if dummy2[cnt].lower().strip() != "abstract":
            cnt += 1
        else:
            cnt += 1
            break
    abstract_string = " ".join(word for word in dummy2[cnt:])
if abstract_string[:1] in string.punctuation or abstract_string[:1] == "—":
    abstract_string = abstract_string[1:]
abstract_string = abstract_string.strip()
print("Abstract:\n\n", abstract_string)

os.remove(pdf)

In [1]:
from abstract_api import abstractextracter

In [2]:
abstractextracter("http://arxiv.org/pdf/1411.6753v1")

'The Dynamic Scalability of resources, a problem in Infrastructure as a Service (IaaS) has been the hotspot for research and industry communities. The heterogeneous and dynamic nature of the Cloud workloads depends on the Quality of Service (QoS) allocation of appropriate workloads to appropriate resources. A workload is an abstraction of work that instance or set of instances that are going to perform. Running a web service or being a Hadoop data node is valid workloads. The efficient management of dynamic nature resources can be done with the help of workloads. Until workload is considered a fundamental capability, the Cloud resources cannot be utilized in an efficient manner. In this paper, different workloads have been identified and categorized along with their characteristics and constraints. The metrics based on Quality of Service (QoS) requirements have been identified for each workload and have been analyzed for creating better application design. Keywords— Cloud Computing; In